In [1]:
%load_ext autoreload
%autoreload 2

from ProblemEvaluator import ProblemEvaluator
import numpy as np
from runner import generate_instance
import matplotlib.pyplot as plt
from optimisation.model.population import Population
from features.RandomWalkAnalysis import RandomWalkAnalysis
from features.GlobalAnalysis import GlobalAnalysis
import time

In [2]:
# Flags for which feature set we want to test.
sample_rw = True
sample_aw = False

# Example problem.
num_samples = 1
num_cores = 1
n_var = 3
problem_name = "DASCMOP1"
problem, _ = generate_instance(problem_name, n_var)
instance_string = f"{problem_name}_d{n_var}"

# Generate evaluator which can generate RW samples and LHS samples.
evaluator = ProblemEvaluator(
    problem, instance_string, "eval", "test_results", num_cores, num_samples
)
evaluator.initialize_number_of_cores()
pre_sampler = evaluator.create_pre_sampler()
pre_sampler.create_pregen_sample_dir()
pre_sampler.create_pops_dir(instance_string, "test_pops")

Initialising evaluator in eval mode.
# name: DASCMOP1
# n_var: 3
# n_obj: 2
# n_ieq_constr: 11
# n_eq_constr: 0

Starting the creation process for population directories for problem: DASCMOP1_d3 in mode: eval
Directory already exists and does not need to be created: test_pops/DASCMOP1_d3/eval/global
Directory already exists and does not need to be created: test_pops/DASCMOP1_d3/eval/rw
Population directories setup complete.
Global population directory: test_pops/DASCMOP1_d3/eval/global
RW population directory: test_pops/DASCMOP1_d3/eval/rw


### Random Walk


In [3]:
sample_number = 1
walk_number = 1

import time  # Import the time module

# Start timing for reading walk neighbours
start_time = time.time()
walk, neighbours = pre_sampler.read_walk_neighbours(sample_number, walk_number)
end_time = time.time()
print(f"Reading walk neighbours took {end_time - start_time:.2f} seconds.")

# Start timing for generating walk neighbour populations

Reading walk neighbours took 0.00 seconds.


In [ ]:
start_time = time.time()
pop_walk, pop_neighbours_list = evaluator.generate_walk_neig_populations(
    problem, walk, neighbours, True
)
end_time = time.time()
print(
    f"Generating walk neighbour populations took {end_time - start_time:.2f} seconds."
)

In [ ]:
# Start timing for saving walk neighbour population
start_time = time.time()
pre_sampler.save_walk_neig_population(
    pop_walk, pop_neighbours_list, sample_number, walk_number
)
end_time = time.time()
print(
    f"Saving walk neighbour population (new method) took {end_time - start_time:.2f} seconds."
)

In [4]:
# Attempt to use pre-generated samples.
start_time = time.time()
pop_walk, pop_neighbours_list = pre_sampler.load_walk_neig_population(
    sample_number, walk_number
)
end_time = time.time()
print(
    f"Loading walk neighbour population (new method) took {end_time - start_time:.2f} seconds."
)

[2024-02-28 13:07:44] Loaded walk and individual neighbours for sample 1, walk 1 from test_pops/DASCMOP1_d3/eval/rw/sample1.
Loading walk neighbour population (new method) took 0.11 seconds.


#### Analysis Object Creation


In [5]:
rw_norm_values = evaluator.compute_rw_normalisation_values(pre_sampler, problem)
rw_analysis = RandomWalkAnalysis(
    pop_walk, pop_neighbours_list, rw_norm_values, evaluator.results_dir
)

[2024-02-28 13:07:46] Initialising normalisation computations for RW samples using 1 cores. This requires full evaluation of the entire sample set and may take some time while still being memory-efficient.
[2024-02-28 13:07:47] Loaded walk and individual neighbours for sample 1, walk 1 from test_pops/DASCMOP1_d3/eval/rw/sample1.
[2024-02-28 13:07:47] Loaded walk and individual neighbours for sample 1, walk 2 from test_pops/DASCMOP1_d3/eval/rw/sample1.
[2024-02-28 13:07:47] Loaded walk and individual neighbours for sample 1, walk 3 from test_pops/DASCMOP1_d3/eval/rw/sample1.
[2024-02-28 13:07:47] Evaluated the normalisation values for this sample set in 1.44 seconds.



#### Running each step of RW feature eval


In [ ]:
start_time = time.time()
rw_analysis.preprocess_nans_on_walks()
end_time = time.time()
print(f"Preprocessing nans on walks took {end_time - start_time:.2f} seconds.")

In [ ]:
# Timing each of the features eval
start_time = time.time()
_ = rw_analysis.compute_solver_related_features()
end_time = time.time()
print(f"RW solver features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = rw_analysis.compute_neighbourhood_distance_features(norm_method="95th")
end_time = time.time()
print(f"RW neighbourhood dist features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = rw_analysis.compute_uncons_neighbourhood_hv_features(norm_method="95th")
end_time = time.time()
print(f"RW uncons neighbourhood HV features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = rw_analysis.compute_cons_neighbourhood_hv_features(norm_method="95th")
end_time = time.time()
print(f"RW cons neighbourhood HV features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = rw_analysis.compute_neighbourhood_violation_features(norm_method="95th")
end_time = time.time()
print(f"RW neighbourhood violation features took {end_time - start_time:.2f} seconds.")

In [14]:
start_time = time.time()
_ = rw_analysis.compute_neighbourhood_dominance_features()
end_time = time.time()
print(f"RW neighbourhood dominance features took {end_time - start_time:.2f} seconds.")

RW neighbourhood dominance features took 0.24 seconds.


In [ ]:
start_time = time.time()
_ = GlobalAnalysis.compute_ic_features(rw_analysis.pop_walk, sample_type="rw")
end_time = time.time()
print(f"RW IC features took {end_time - start_time:.2f} seconds.")

### Global


In [ ]:
sample_number = 1
# Start timing for reading walk neighbours
start_time = time.time()
global_sample = pre_sampler.read_global_sample(sample_number)
end_time = time.time()
print(f"Reading global sample took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
pop_global = evaluator.generate_global_population(problem, global_sample, True)
end_time = time.time()
print(f"Generating global populations took {end_time - start_time:.2f} seconds.")

In [ ]:
# Start timing for saving walk neighbour population
start_time = time.time()
pre_sampler.save_global_population(pop_global, sample_number)
end_time = time.time()
print(
    f"Saving global population (new method) took {end_time - start_time:.2f} seconds."
)

In [ ]:
# Attempt to use pre-generated samples.
start_time = time.time()
pop_global = pre_sampler.load_global_population(sample_number)
end_time = time.time()
print(
    f"Loading global population (new method) took {end_time - start_time:.2f} seconds."
)

#### Analysis Object Creation


In [ ]:
global_norm_values = evaluator.compute_global_normalisation_values(pre_sampler, problem)
global_analysis = GlobalAnalysis(pop_global, global_norm_values, evaluator.results_dir)

In [ ]:
print(global_analysis.pop.extract_var().shape)
print(global_analysis.pop.extract_obj().shape)
print(global_analysis.pop.extract_rank().shape)
print(global_analysis.pop.extract_uncons_rank().shape)
print(global_analysis.pop.extract_cv().shape)
print(global_analysis.pop.extract_cons().shape)

#### Running each step of Global feature eval


In [ ]:
from features.Analysis import Analysis

global_analysis.pop, _ = global_analysis.pop.remove_nan_inf_rows(
    "global", re_evaluate=True
)

In [ ]:
start_time = time.time()
_ = global_analysis.compute_fsr()
end_time = time.time()
print(f"Global feasibility features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.corr_obj()
end_time = time.time()
print(f"Global obj corr features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.obj_kurt()
_ = global_analysis.obj_skew()
end_time = time.time()
print(f"Global obj skew, kurtosis features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.uc_rk_distr()
end_time = time.time()
print(f"Global uc rk distr features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.cv_distr(norm_method="95th")
end_time = time.time()
print(f"Global cv distr features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.PiIZ()
end_time = time.time()
print(f"Global ideal zone features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.compute_ps_pf_distances(norm_method="95th")
end_time = time.time()
print(f"Global PS PF distance features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.compute_PF_UPF_features(norm_method="95th")
end_time = time.time()
print(f"Global PF UPF features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.dist_corr()
end_time = time.time()
print(f"Global dist corr features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.compute_ranks_cv_corr()
end_time = time.time()
print(f"Global ranks cv corr features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.rk_uc_var_mdl()
end_time = time.time()
print(f"Global rk uc var mdl features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = global_analysis.cv_var_mdl()
end_time = time.time()
print(f"Global cv var mdl features took {end_time - start_time:.2f} seconds.")

In [ ]:
start_time = time.time()
_ = GlobalAnalysis.compute_ic_features(global_analysis.pop, sample_type="global")
end_time = time.time()
print(f"Global IC features took {end_time - start_time:.2f} seconds.")

## Testing NaNs and such


In [17]:
sample_number = 1
# Start timing for reading walk neighbours
start_time = time.time()
global_sample = pre_sampler.read_global_sample(sample_number)
end_time = time.time()
print(f"Reading global sample took {end_time - start_time:.2f} seconds.")

# Inject a couple of nans into the sample.
global_sample[1, :] = np.nan
global_sample[3, :] = np.nan

Global sample size for 3d, no. 1: 3000
Reading global sample took 0.00 seconds.


array([[0.16791331, 0.01359483, 0.82458498],
       [       nan,        nan,        nan],
       [0.22121766, 0.71009222, 0.1054732 ],
       ...,
       [0.10760741, 0.88157479, 0.85282529],
       [0.15204733, 0.40958643, 0.48208943],
       [0.93609429, 0.9556829 , 0.98304957]])

In [ ]:
start_time = time.time()
pop_global = evaluator.generate_global_population(problem, global_sample, True)
end_time = time.time()
print(f"Generating global populations took {end_time - start_time:.2f} seconds.")